In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

classes = {
    "Plate1": 0,
    "Plate3": 1
}

object_name = "Plate3"
folder = "obb/train"

image_files = glob.glob(f"./datasets/{folder}/images/image_{object_name}_*.jpg")

for file in image_files:
    print(file, file.replace("/images\\", "/labels\\").replace(".jpg", ".txt"))

    image = cv2.imread(file)
    image[:200,:] = image[0,0]
    image[:,1750:] = image[0,0]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5, 5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    min_area = 5000
    max_area = 200000
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area and cv2.contourArea(cnt) < max_area]

    image_filtered = image.copy()

    for contour in filtered_contours:
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.array(box, dtype=np.int32)
        cv2.drawContours(image_filtered, [box], 0, (0, 255, 0), 1)

    plt.figure()
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(image_filtered, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    yolo_obb_annotations = []
    for contour in filtered_contours:
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        print(box)
        box[:, 0] /= 1920
        box[:, 1] /= 1440

        annotation = f"{classes[object_name]} {' '.join(f'{x:.6f} {y:.6f}' for x, y in box)}"
        yolo_obb_annotations.append(annotation)

    with open(file.replace("/images\\", "/labels\\").replace(".jpg", ".txt"), "w") as f:
        f.write("\n".join(yolo_obb_annotations))
